### 練習問題2.68
encode手続きは、引数としてメッセージと木を取り、  
メッセージを符号化したビットのリストを返す。

    (define (encode message tree)
        (if (null? message) '()
            (append
                (encode-symbol (car message) tree)
                (encode (cdr message) tree))
        )
    ) 

encode-symbolは、与えられた木に従って与えられた記号を符号化したビット列を返す。  
この手続きを書け。  
記号が木に含まれていなければエラーを出すようにencode-symbolを設計せよ。  
練習問題2.67で得た結果をサンプルの木を使って符号化し、  
それが元のサンプルメッセージと同じになるかどうか確認して、  
書いた手続きをテストせよ。

In [1]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

; コンストラクタ
(define (make-code-tree left right)
  (list left right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))
    )
  )

; セレクタ
(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree) (list (symbol-leaf tree))
      (caddr tree)
    )
  )
(define (weight tree)
  (if (leaf? tree) (weight-leaf tree)
      (cadddr tree)
    )
  )

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits) '()
        (let ((next-branch (choose-branch (car bits) current-branch)))
          (if (leaf? next-branch) (cons (symbol-leaf next-branch) (decode-1 (cdr bits) tree))
              (decode-1 (cdr bits) next-branch))
          )
    )
    )
  (decode-1 bits tree)
)  
(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))
        )
  )

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(define (make-leaf-set pairs)
  (if (null? pairs) '()
      (let ((pair (car pairs)))
        (adjoin-set (make-leaf (car pair) ; symbol
                               (cadr pair)) ; weight
                    (make-leaf-set (cdr pairs)))
        )
    )
  )

In [2]:
(define sample-tree
    (make-code-tree (make-leaf 'A 4)
        (make-code-tree (make-leaf 'B 2)
            (make-code-tree (make-leaf 'D 1)
                (make-leaf 'C 1)
            )
        )
    )
)
(define sample-message '(0 1 1 0 0 1 0 1 0 1 1 1 0))

In [3]:
; 復号
(decode sample-message sample-tree)

(A D A B B C A)

In [4]:
; ハフマン符号化木の確認
sample-tree

((leaf A 4) ((leaf B 2) ((leaf D 1) (leaf C 1) (D C) 2) (B D C) 4) (A B D C) 8)

In [5]:
; 符号化処理
; 回答
(define (encode-symbol symbol tree)
  (define (iter sub result)
    (if (leaf? sub)
        (if (eq? (symbol-leaf sub) symbol) result
            '())
        (let ((l (left-branch sub))
              (r (right-branch sub)))
          (let ((l-result (iter l (append result '(0)))))
            (if (not (null? l-result)) l-result
                (iter r (append result '(1)))
              )
            )
        )
      )
    )
  (let ((result (iter tree '())))
    (if (null? result) (error "bad symbol: ENCODE" symbol)
        result
        )
    )
)

In [6]:
; 個々の記号で動作確認
(display (encode-symbol 'A sample-tree))
(newline)
(display (encode-symbol 'B sample-tree))
(newline)
(display (encode-symbol 'C sample-tree))
(newline)
(display (encode-symbol 'D sample-tree))
(newline)
;(display (encode-symbol 'E sample-tree))
;(newline)

(0)
(1 0)
(1 1 1)
(1 1 0)


In [7]:
; 問題の符号化処理 
(define (encode message tree)
    (if (null? message) '()
        (append
            (encode-symbol (car message) tree)
            (encode (cdr message) tree))
    )
)

In [8]:
(encode '(A D A B B C A) sample-tree)

(0 1 1 0 0 1 0 1 0 1 1 1 0)